# MOVIES PROJECT

necesary imports

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

get responses from lots of urls <br>
starting with: <br>
https://www.imdb.com/...&start=1 <br>
up to: <br>
https://www.imdb.com/...&start=6651

In [40]:
responses = []
for position in range(1, 6652, 50):
    request = requests.get(
        f"https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&countries=us&sort=user_rating,desc&start={position}"
    )
    responses.append(request)
len(responses)

2

sopify all of them

In [41]:
soups = []
for response in responses:
    soups.append(BeautifulSoup(response.text, 'html.parser'))
len(soups)


2

## RELEASE DATES

date cleaning function

In [42]:
def extract_int(string):
    return re.search(r"\d+", string).group()

traverse the soups in search for dates, create a list of them, serialise it

In [43]:
movie_release_dates = []
for soup in soups:
    movie_release_dates += soup.findAll('span', class_= 'lister-item-year text-muted unbold')

list_release_dates = []
for str_date in movie_release_dates:
    list_release_dates.append( extract_int(str_date.text) )

rel_dates_series = pd.Series(list_release_dates, name="Release Date")
rel_dates_series

0     1994
1     1972
2     2008
3     1974
4     1957
      ... 
95    2018
96    2019
97    2010
98    2009
99    2013
Name: Release Date, Length: 100, dtype: object

## RUNTIMES

create a time-cleaning function

In [44]:
def clean_time(time_string):

    time_string = time_string.split()
    time_string = time_string[0]

    return int(time_string)

traverse the soups in search for runtimes, create a list of them, serialise it

In [45]:
runtimes = []
for soup in soups:
    runtimes += soup.findAll('span', class_="runtime")

movie_runtimes = []
for runtime in runtimes:
    movie_runtimes.append( clean_time(runtime.text) )
runtimes_series = pd.Series(movie_runtimes, name="Runtime")

runtimes_series

0     142
1     175
2     152
3     202
4      96
     ... 
95    130
96     96
97    138
98     96
99    180
Name: Runtime, Length: 100, dtype: int64

## MOVIE NAMES

traverse the soups in search for movie names, create a list of them, serialise it

In [46]:
movie_names = []
for soup in soups:
    movie_names += soup.find_all(name="h3", class_="lister-item-header")

names_list = []
for mn in movie_names:
    names_list.append(list(mn)[3].text)

names_series = pd.Series(names_list, name="Movie name")
names_series

0         Skazani na Shawshank
1             Ojciec chrzestny
2               Mroczny Rycerz
3          Ojciec chrzestny II
4     Dwunastu gniewnych ludzi
                ...           
95                  Green Book
96                       Klaus
97              Wyspa tajemnic
98                       Odlot
99          Wilk z Wall Street
Name: Movie name, Length: 100, dtype: object

## MOVIE TYPES

traverse the soups in search for movie names, clean them, create a list of them and serialise it

In [47]:
genres = []

for soup in soups:
    genres += soup.find_all(name="span", class_="genre")

genres_list = []
for gen in genres:
    string = list(gen)[0].replace("\n", "").strip()
    genres_list.append(string)

genres_series = pd.Series(genres_list, name="Genre")
genres_series

0                            Drama
1                     Crime, Drama
2             Action, Crime, Drama
3                     Crime, Drama
4                     Crime, Drama
                  ...             
95        Biography, Comedy, Drama
96    Animation, Adventure, Comedy
97               Mystery, Thriller
98    Animation, Adventure, Comedy
99         Biography, Crime, Drama
Name: Genre, Length: 100, dtype: object

## RATING

traverse the soup, get stuff from it and serialise it in 2 steps

In [48]:
# <div class="inline-block ratings-imdb-rating" name="ir" data-value="8,3">
ratings = []
for soup in soups:
    ratings += soup.find_all(name="div", attrs={'name':'ir'})

ratings_series = pd.Series([ float(rt.text.strip()) for rt in ratings ], name="Rating")
ratings_series

0     9.3
1     9.2
2     9.0
3     9.0
4     9.0
     ... 
95    8.2
96    8.2
97    8.2
98    8.2
99    8.2
Name: Rating, Length: 100, dtype: float64

## PROFIT!

In [49]:
all_series = [names_series, ratings_series, rel_dates_series, runtimes_series, genres_series]

df = pd.concat(all_series, axis=1)

# %timeit df.loc[df.loc[:,'Genre'].str.contains('Action')]
# %timeit df.loc[df['Genre'].str.contains('Action')]
# %timeit df.loc[ [('Action' in genre) for genre in df.loc[:,'Genre']]]
# %timeit df.loc[ [('Action' in genre) for genre in df['Genre']]]


In [50]:
df

,Movie name,Rating,Release Date,Runtime,Genre
0,Skazani na Shawshank,9.3,1994,142,Drama
1,Ojciec chrzestny,9.2,1972,175,"Crime, Drama"
2,Mroczny Rycerz,9.0,2008,152,"Action, Crime, Drama"
3,Ojciec chrzestny II,9.0,1974,202,"Crime, Drama"
4,Dwunastu gniewnych ludzi,9.0,1957,96,"Crime, Drama"
...,...,...,...,...,...
95,Green Book,8.2,2018,130,"Biography, Comedy, Drama"
96,Klaus,8.2,2019,96,"Animation, Adventure, Comedy"
97,Wyspa tajemnic,8.2,2010,138,"Mystery, Thriller"
98,Odlot,8.2,2009,96,"Animation, Adventure, Comedy"


extract action movies

In [51]:
action = df.loc[ [('Action' in genre) for genre in df['Genre']]]
action

,Movie name,Rating,Release Date,Runtime,Genre
2,Mroczny Rycerz,9.0,2008,152,"Action, Crime, Drama"
5,Władca pierścieni: Powrót króla,8.9,2003,201,"Action, Adventure, Drama"
8,Incepcja,8.8,2010,148,"Action, Adventure, Sci-Fi"
10,Władca pierścieni: Drużyna Pierścienia,8.8,2001,178,"Action, Adventure, Drama"
12,Władca pierścieni: Dwie wieże,8.7,2002,179,"Action, Adventure, Drama"
13,Matrix,8.7,1999,136,"Action, Sci-Fi"
15,Gwiezdne wojny: Część V - Imperium kontratakuje,8.7,1980,124,"Action, Adventure, Fantasy"
22,Gwiezdne wojny,8.6,1977,121,"Action, Adventure, Fantasy"
30,Gladiator,8.5,2000,155,"Action, Adventure, Drama"
33,Léon zawodowiec,8.5,1994,110,"Action, Crime, Drama"
